In [1]:
!pip install ISLP

  Using cached ISLP-0.4.0-py3-none-any.whl.metadata (7.0 kB)
  Using cached lifelines-0.30.0-py3-none-any.whl.metadata (3.2 kB)
  Using cached pygam-0.9.1-py3-none-any.whl.metadata (7.1 kB)
  Using cached torch-2.5.1-cp312-cp312-win_amd64.whl.metadata (28 kB)
  Using cached pytorch_lightning-2.4.0-py3-none-any.whl.metadata (21 kB)
  Using cached torchmetrics-1.5.2-py3-none-any.whl.metadata (20 kB)
  Using cached autograd-1.7.0-py3-none-any.whl.metadata (7.5 kB)
  Using cached autograd_gamma-0.5.0-py3-none-any.whl
  Using cached formulaic-1.0.2-py3-none-any.whl.metadata (6.8 kB)
  Using cached progressbar2-4.5.0-py3-none-any.whl.metadata (16 kB)
  Using cached scipy-1.11.4-cp312-cp312-win_amd64.whl.metadata (60 kB)
  Using cached lightning_utilities-0.11.8-py3-none-any.whl.metadata (5.2 kB)
  Using cached sympy-1.13.1-py3-none-any.whl.metadata (12 kB)
  Using cached interface_meta-1.3.0-py3-none-any.whl.metadata (6.7 kB)
  Using cached python_utils-3.9.0-py2.py3-none-any.whl.metadata (9

In [54]:
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
import pandas as pd
import numpy as np
import os
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.metrics import r2_score, mean_squared_error, confusion_matrix, accuracy_score, roc_curve, roc_auc_score,log_loss,classification_report
from sklearn.model_selection import train_test_split, KFold, cross_val_score, GridSearchCV
from sklearn.preprocessing import PolynomialFeatures, LabelEncoder, OneHotEncoder,MinMaxScaler, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import ElasticNet, LogisticRegression, LinearRegression
from sklearn.compose import make_column_transformer, make_column_selector
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import BernoulliNB, GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import plot_tree
from sklearn.tree import DecisionTreeRegressor




In [56]:
from ISLP import load_data
Credit = load_data('Credit')


In [58]:
Credit.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 12 columns):
 #   Column     Non-Null Count  Dtype   
---  ------     --------------  -----   
 0   ID         400 non-null    int64   
 1   Income     400 non-null    float64 
 2   Limit      400 non-null    int64   
 3   Rating     400 non-null    int64   
 4   Cards      400 non-null    int64   
 5   Age        400 non-null    int64   
 6   Education  400 non-null    int64   
 7   Gender     400 non-null    category
 8   Student    400 non-null    category
 9   Married    400 non-null    category
 10  Ethnicity  400 non-null    category
 11  Balance    400 non-null    int64   
dtypes: category(4), float64(1), int64(7)
memory usage: 27.2 KB


In [60]:
df = Credit.drop('ID',axis=1)

In [62]:
y = df['Balance']
X = df.drop('Balance',axis=1 )

In [64]:
X.columns

Index(['Income', 'Limit', 'Rating', 'Cards', 'Age', 'Education', 'Gender',
       'Student', 'Married', 'Ethnicity'],
      dtype='object')

In [66]:
X_train,X_test,y_train,y_test = train_test_split(X, y, random_state = 24, test_size = 0.3)

In [124]:
ohe = OneHotEncoder(handle_unknown = 'ignore', sparse_output = False).set_output(transform = 'pandas')
ct = make_column_transformer(('passthrough',make_column_selector(dtype_exclude = 'category') ),
                                  (ohe,make_column_selector(dtype_include = 'category') ),
                                  verbose_feature_names_out = False).set_output(transform = 'pandas')



el = ElasticNet(random_state = 24)
pipe_el = Pipeline([('CT',ct),('EL',el)])
pipe_el.fit(X_train,y_train)
y_pred = pipe_el.predict(X_test)
r2_score(y_test,y_pred)

C:\Users\dai.STUDENTSDC\AppData\Local\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:678: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.054e+06, tolerance: 6.154e+03
  model = cd_fast.enet_coordinate_descent(


0.9057359592636075

In [126]:
dtc = DecisionTreeRegressor(random_state = 24)
pipe_dtc = Pipeline([('CT',ct),('DT',dtc)])

pipe_dtc.fit(X_train,y_train)
y_pred = pipe_dtc.predict(X_test)
r2_score(y_test,y_pred)

0.899162982269506

In [136]:
ohe = OneHotEncoder(handle_unknown = 'ignore', sparse_output = False).set_output(transform = 'pandas')
ct = make_column_transformer(('passthrough',make_column_selector(dtype_exclude = 'category') ),
                                  (ohe,make_column_selector(dtype_include = 'category') ),
                                  verbose_feature_names_out = False).set_output(transform = 'pandas')
knn = KNeighborsRegressor()
pipe_kn = Pipeline([('CT',ct),('KNN',knn)])


pipe_kn.fit(X_train,y_train)
y_pred = pipe_kn.predict(X_test)
r2_score(y_test,y_pred)

0.7899310404076199

In [132]:
from sklearn.ensemble import RandomForestClassifier, VotingClassifier,VotingRegressor
voting = VotingRegressor([('EL',pipe_el),('DT',pipe_dtc),('KNN',pipe_kn)])
voting.fit(X_train,y_train)
y_pred = voting.predict(X_test)
r2_score(y_test,y_pred)

C:\Users\dai.STUDENTSDC\AppData\Local\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:678: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.054e+06, tolerance: 6.154e+03
  model = cd_fast.enet_coordinate_descent(


0.914557198194179

In [158]:
voting = VotingRegressor([('EL',pipe_el),('DT',pipe_dtc),('KNN',pipe_kn)],
                        weights = [8,5,0])
voting.fit(X_train,y_train)
y_pred = voting.predict(X_test)
r2_score(y_test,y_pred)

C:\Users\dai.STUDENTSDC\AppData\Local\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:678: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.054e+06, tolerance: 6.154e+03
  model = cd_fast.enet_coordinate_descent(


0.9318631159191757

## One time encoding with Pipe without weight in voting

In [168]:
ohe = OneHotEncoder(handle_unknown = 'ignore', sparse_output = False).set_output(transform = 'pandas')
ct = make_column_transformer(('passthrough',make_column_selector(dtype_exclude = 'category') ),
                                  (ohe,make_column_selector(dtype_include = 'category') ),
                                  verbose_feature_names_out = False).set_output(transform = 'pandas')

el = ElasticNet(random_state = 24)
dtc = DecisionTreeRegressor(random_state = 24)
knn = KNeighborsRegressor()
voting = VotingRegressor([('EL',el),('DT',dtc),('KNN',knn)]
                        )
pipe = Pipeline([('CT',ct),('VT',voting)])
pipe.fit(X_train,y_train)

C:\Users\dai.STUDENTSDC\AppData\Local\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:678: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.054e+06, tolerance: 6.154e+03
  model = cd_fast.enet_coordinate_descent(


Pipeline(steps=[('CT',
                 ColumnTransformer(transformers=[('passthrough', 'passthrough',
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x0000014AB8682540>),
                                                 ('onehotencoder',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse_output=False),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x0000014AB86BAB10>)],
                                   verbose_feature_names_out=False)),
                ('VT',
                 VotingRegressor(estimators=[('EL',
                                              ElasticNet(random_state=24)),
                                             ('DT',
                                              DecisionTreeRegressor(random_state=24)),
                                             ('KNN', KNeighborsRegressor())]))])

In [170]:
y_pred = pipe.predict(X_test)
r2_score(y_test,y_pred)

0.914557198194179

## With Weight Voting 

In [174]:
ohe = OneHotEncoder(handle_unknown = 'ignore', sparse_output = False).set_output(transform = 'pandas')
ct = make_column_transformer(('passthrough',make_column_selector(dtype_exclude = 'category') ),
                                  (ohe,make_column_selector(dtype_include = 'category') ),
                                  verbose_feature_names_out = False).set_output(transform = 'pandas')

el = ElasticNet(random_state = 24)
dtc = DecisionTreeRegressor(random_state = 24)
knn = KNeighborsRegressor()
voting = VotingRegressor([('EL',el),('DT',dtc),('KNN',knn)],
                         weights = [8,5,0]
                        )
pipe = Pipeline([('CT',ct),('VT',voting)])
pipe.fit(X_train,y_train)

C:\Users\dai.STUDENTSDC\AppData\Local\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:678: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.054e+06, tolerance: 6.154e+03
  model = cd_fast.enet_coordinate_descent(


Pipeline(steps=[('CT',
                 ColumnTransformer(transformers=[('passthrough', 'passthrough',
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x0000014AB86B85F0>),
                                                 ('onehotencoder',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse_output=False),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x0000014AB86B8170>)],
                                   verbose_feature_names_out=False)),
                ('VT',
                 VotingRegressor(estimators=[('EL',
                                              ElasticNet(random_state=24)),
                                             ('DT',
                                              DecisionTreeRegressor(random_state=24)),
                                             ('KNN', KNeighborsRegressor())],
                                 weights=[8, 5, 0]))])

In [176]:
y_pred = pipe.predict(X_test)
r2_score(y_test,y_pred)

0.9318631159191757

In [180]:
pipe.get_params()

{'memory': None,
 'steps': [('CT',
   ColumnTransformer(transformers=[('passthrough', 'passthrough',
                                    <sklearn.compose._column_transformer.make_column_selector object at 0x0000014AB86B85F0>),
                                   ('onehotencoder',
                                    OneHotEncoder(handle_unknown='ignore',
                                                  sparse_output=False),
                                    <sklearn.compose._column_transformer.make_column_selector object at 0x0000014AB86B8170>)],
                     verbose_feature_names_out=False)),
  ('VT',
   VotingRegressor(estimators=[('EL', ElasticNet(random_state=24)),
                               ('DT', DecisionTreeRegressor(random_state=24)),
                               ('KNN', KNeighborsRegressor())],
                   weights=[8, 5, 0]))],
 'verbose': False,
 'CT': ColumnTransformer(transformers=[('passthrough', 'passthrough',
                                  <sklea

In [182]:
scaler_std = StandardScaler()
scaler_mm = MinMaxScaler()
kfold = KFold(n_splits = 5, shuffle = True,random_state = 24)


In [215]:
params = {'VT__KNN__n_neighbors': [2,5,7],
          'VT__EL__alpha': np.linspace(0.001,3,3),
          'VT__DT__max_depth': [None,3,5],
          'VT__DT__min_samples_leaf': [1,5,10],
          'VT__DT__min_samples_split': [2,5,10],
          'VT__weights': [[6,6,3],[7,7,2],[9,9,1]]
         }


gcv = GridSearchCV(pipe,param_grid = params,verbose = 3, cv = kfold) 
gcv.fit(X,y)

Fitting 5 folds for each of 729 candidates, totalling 3645 fits
[CV 1/5] END VT__DT__max_depth=None, VT__DT__min_samples_leaf=1, VT__DT__min_samples_split=2, VT__EL__alpha=0.001, VT__KNN__n_neighbors=2, VT__weights=[6, 6, 3];, score=0.947 total time=   0.0s
[CV 2/5] END VT__DT__max_depth=None, VT__DT__min_samples_leaf=1, VT__DT__min_samples_split=2, VT__EL__alpha=0.001, VT__KNN__n_neighbors=2, VT__weights=[6, 6, 3];, score=0.870 total time=   0.0s
[CV 3/5] END VT__DT__max_depth=None, VT__DT__min_samples_leaf=1, VT__DT__min_samples_split=2, VT__EL__alpha=0.001, VT__KNN__n_neighbors=2, VT__weights=[6, 6, 3];, score=0.926 total time=   0.0s
[CV 4/5] END VT__DT__max_depth=None, VT__DT__min_samples_leaf=1, VT__DT__min_samples_split=2, VT__EL__alpha=0.001, VT__KNN__n_neighbors=2, VT__weights=[6, 6, 3];, score=0.949 total time=   0.0s
[CV 5/5] END VT__DT__max_depth=None, VT__DT__min_samples_leaf=1, VT__DT__min_samples_split=2, VT__EL__alpha=0.001, VT__KNN__n_neighbors=2, VT__weights=[6, 6, 3]

C:\Users\dai.STUDENTSDC\AppData\Local\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:678: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.564e+06, tolerance: 6.884e+03
  model = cd_fast.enet_coordinate_descent(
C:\Users\dai.STUDENTSDC\AppData\Local\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:678: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.459e+06, tolerance: 6.842e+03
  model = cd_fast.enet_coordinate_descent(
C:\Users\dai.STUDENTSDC\AppData\Local\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:678: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or

[CV 2/5] END VT__DT__max_depth=None, VT__DT__min_samples_leaf=1, VT__DT__min_samples_split=2, VT__EL__alpha=0.001, VT__KNN__n_neighbors=2, VT__weights=[7, 7, 2];, score=0.875 total time=   0.0s
[CV 3/5] END VT__DT__max_depth=None, VT__DT__min_samples_leaf=1, VT__DT__min_samples_split=2, VT__EL__alpha=0.001, VT__KNN__n_neighbors=2, VT__weights=[7, 7, 2];, score=0.932 total time=   0.0s
[CV 4/5] END VT__DT__max_depth=None, VT__DT__min_samples_leaf=1, VT__DT__min_samples_split=2, VT__EL__alpha=0.001, VT__KNN__n_neighbors=2, VT__weights=[7, 7, 2];, score=0.952 total time=   0.0s
[CV 5/5] END VT__DT__max_depth=None, VT__DT__min_samples_leaf=1, VT__DT__min_samples_split=2, VT__EL__alpha=0.001, VT__KNN__n_neighbors=2, VT__weights=[7, 7, 2];, score=0.950 total time=   0.0s
[CV 1/5] END VT__DT__max_depth=None, VT__DT__min_samples_leaf=1, VT__DT__min_samples_split=2, VT__EL__alpha=0.001, VT__KNN__n_neighbors=2, VT__weights=[9, 9, 1];, score=0.955 total time=   0.0s
[CV 2/5] END VT__DT__max_depth

C:\Users\dai.STUDENTSDC\AppData\Local\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:678: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.580e+06, tolerance: 6.921e+03
  model = cd_fast.enet_coordinate_descent(
C:\Users\dai.STUDENTSDC\AppData\Local\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:678: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.510e+06, tolerance: 6.558e+03
  model = cd_fast.enet_coordinate_descent(
C:\Users\dai.STUDENTSDC\AppData\Local\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:678: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or

[CV 4/5] END VT__DT__max_depth=None, VT__DT__min_samples_leaf=1, VT__DT__min_samples_split=2, VT__EL__alpha=0.001, VT__KNN__n_neighbors=2, VT__weights=[9, 9, 1];, score=0.953 total time=   0.0s
[CV 5/5] END VT__DT__max_depth=None, VT__DT__min_samples_leaf=1, VT__DT__min_samples_split=2, VT__EL__alpha=0.001, VT__KNN__n_neighbors=2, VT__weights=[9, 9, 1];, score=0.953 total time=   0.0s
[CV 1/5] END VT__DT__max_depth=None, VT__DT__min_samples_leaf=1, VT__DT__min_samples_split=2, VT__EL__alpha=0.001, VT__KNN__n_neighbors=5, VT__weights=[6, 6, 3];, score=0.952 total time=   0.0s
[CV 2/5] END VT__DT__max_depth=None, VT__DT__min_samples_leaf=1, VT__DT__min_samples_split=2, VT__EL__alpha=0.001, VT__KNN__n_neighbors=5, VT__weights=[6, 6, 3];, score=0.868 total time=   0.0s
[CV 3/5] END VT__DT__max_depth=None, VT__DT__min_samples_leaf=1, VT__DT__min_samples_split=2, VT__EL__alpha=0.001, VT__KNN__n_neighbors=5, VT__weights=[6, 6, 3];, score=0.931 total time=   0.0s
[CV 4/5] END VT__DT__max_depth

...

C:\Users\dai.STUDENTSDC\AppData\Local\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:678: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.716e+06, tolerance: 6.884e+03
  model = cd_fast.enet_coordinate_descent(
C:\Users\dai.STUDENTSDC\AppData\Local\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:678: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.192e+06, tolerance: 6.842e+03
  model = cd_fast.enet_coordinate_descent(
C:\Users\dai.STUDENTSDC\AppData\Local\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:678: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or

[CV 4/5] END VT__DT__max_depth=5, VT__DT__min_samples_leaf=10, VT__DT__min_samples_split=10, VT__EL__alpha=3.0, VT__KNN__n_neighbors=7, VT__weights=[6, 6, 3];, score=0.889 total time=   0.0s
[CV 5/5] END VT__DT__max_depth=5, VT__DT__min_samples_leaf=10, VT__DT__min_samples_split=10, VT__EL__alpha=3.0, VT__KNN__n_neighbors=7, VT__weights=[6, 6, 3];, score=0.899 total time=   0.0s
[CV 1/5] END VT__DT__max_depth=5, VT__DT__min_samples_leaf=10, VT__DT__min_samples_split=10, VT__EL__alpha=3.0, VT__KNN__n_neighbors=7, VT__weights=[7, 7, 2];, score=0.907 total time=   0.0s
[CV 2/5] END VT__DT__max_depth=5, VT__DT__min_samples_leaf=10, VT__DT__min_samples_split=10, VT__EL__alpha=3.0, VT__KNN__n_neighbors=7, VT__weights=[7, 7, 2];, score=0.857 total time=   0.0s
[CV 3/5] END VT__DT__max_depth=5, VT__DT__min_samples_leaf=10, VT__DT__min_samples_split=10, VT__EL__alpha=3.0, VT__KNN__n_neighbors=7, VT__weights=[7, 7, 2];, score=0.838 total time=   0.0s
[CV 4/5] END VT__DT__max_depth=5, VT__DT__min

C:\Users\dai.STUDENTSDC\AppData\Local\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:678: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.360e+06, tolerance: 6.511e+03
  model = cd_fast.enet_coordinate_descent(
C:\Users\dai.STUDENTSDC\AppData\Local\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:678: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.716e+06, tolerance: 6.884e+03
  model = cd_fast.enet_coordinate_descent(
C:\Users\dai.STUDENTSDC\AppData\Local\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:678: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or

[CV 2/5] END VT__DT__max_depth=5, VT__DT__min_samples_leaf=10, VT__DT__min_samples_split=10, VT__EL__alpha=3.0, VT__KNN__n_neighbors=7, VT__weights=[9, 9, 1];, score=0.863 total time=   0.0s
[CV 3/5] END VT__DT__max_depth=5, VT__DT__min_samples_leaf=10, VT__DT__min_samples_split=10, VT__EL__alpha=3.0, VT__KNN__n_neighbors=7, VT__weights=[9, 9, 1];, score=0.833 total time=   0.0s
[CV 4/5] END VT__DT__max_depth=5, VT__DT__min_samples_leaf=10, VT__DT__min_samples_split=10, VT__EL__alpha=3.0, VT__KNN__n_neighbors=7, VT__weights=[9, 9, 1];, score=0.899 total time=   0.0s
[CV 5/5] END VT__DT__max_depth=5, VT__DT__min_samples_leaf=10, VT__DT__min_samples_split=10, VT__EL__alpha=3.0, VT__KNN__n_neighbors=7, VT__weights=[9, 9, 1];, score=0.908 total time=   0.0s


GridSearchCV(cv=KFold(n_splits=5, random_state=24, shuffle=True),
             estimator=Pipeline(steps=[('CT',
                                        ColumnTransformer(transformers=[('passthrough',
                                                                         'passthrough',
                                                                         <sklearn.compose._column_transformer.make_column_selector object at 0x0000014AB86B85F0>),
                                                                        ('onehotencoder',
                                                                         OneHotEncoder(handle_unknown='ignore',
                                                                                       sparse_output=False),
                                                                         <sklearn.compose._column_transfo...
                                                                    ('DT',
                                                                     DecisionTreeRegressor(random_state=24)),
                                                                    ('KNN',
                                                                     KNeighborsRegressor())],
                                                        weights=[8, 5, 0]))]),
             param_grid={'VT__DT__max_depth': [None, 3, 5],
                         'VT__DT__min_samples_leaf': [1, 5, 10],
                         'VT__DT__min_samples_split': [2, 5, 10],
                         'VT__EL__alpha': array([1.0000e-03, 1.5005e+00, 3.0000e+00]),
                         'VT__KNN__n_neighbors': [2, 5, 7],
                         'VT__weights': [[6, 6, 3], [7, 7, 2], [9, 9, 1]]},
             verbose=3)

In [ ]:
print(gcv.best_params_)
print(gcv.best_score_)

In [206]:
pd_cv = pd.DataFrame(gcv.cv_results_)
pd_cv.to_csv("gridSearchResults.csv",index = False)

In [210]:
pd_cv.shape

(729, 19)